# Scratch code for testing purposes and small experiments. 


In [1]:

import numpy as np
import importlib
from Graph import Graph
from helpers import *
import pandas as pd
import random
from tqdm import tqdm
from deterministic_attack import DeterministicAttack
from revisited_spectral import RevisitedSpectral
from erdos import SpectralAttack


%load_ext autoreload
%autoreload 2

# Exploring the spectral approach

In [2]:
dataset = "netscience"
G = Graph.from_txt(f"datasets/{dataset}.txt")
G1, G2 = G.split_dataset(common_prop=0.0, graph1_prop=0.0)
A = np.dot(G.adj_matrix, G.adj_matrix.T)

## Deterministic -> Heuristic -> Deterministic 

In [3]:
deter = DeterministicAttack(G1, A)
deter.run()
Gstar = deter.get_reconstructed_graph()
known_edges = Gstar.edges()
proba = RevisitedSpectral(Gstar, A)
n_nodes = Gstar.adj_matrix.shape[0]
proba.run(alpha=0.75, beta=0.25, gamma=0)
Gstar = proba.get_reconstructed_graph()
print("Errors after revisited:", np.sum(np.abs(Gstar.adj_matrix - G.adj_matrix)))
stats = ROC_stats(Gstar, G)
print("Before sanity check:", stats)
forgotten_slots = proba.sanity_check_with_high_loss()
Gstar = proba.get_reconstructed_graph()
stats = ROC_stats(Gstar, G)
print("After sanity check:", stats)

deter = DeterministicAttack(Gstar, A)
deter.run()
Gstar = deter.get_reconstructed_graph()
Gstar_fixed = Gstar.copy()
Gstar_fixed.fix_edges()
print("Errors :", np.sum(np.abs(Gstar_fixed.adj_matrix - G.adj_matrix)))




Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 18359.39it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  |  3051 |
| Number of reconstructed edges |   90  |
|    Number of unknown edges    | 68869 |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 17001.64it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 22548 |
| Number of reconstructed edges |   90  |
|    Number of unknown edges    | 49372 |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 21231.01it/s]


...

+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71068 |
| Number of reconstructed edges |  908  |
|    Number of unknown edges    |   34  |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 908/908 [00:00<00:00, 33436.30it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71084 |
| Number of reconstructed edges |  908  |
|    Number of unknown edges    |   18  |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 908/908 [00:00<00:00, 32114.24it/s]

+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71084 |
| Number of reconstructed edges |  908  |
|    Number of unknown edges    |   18  |
+-------------------------------+-------+ 

Errors : 12


## Erdos method

In [4]:
proba = SpectralAttack(A)
proba.run()
Gstar = proba.get_reconstructed_graph()
print("Errors after erdos:", np.sum(np.abs(Gstar.adj_matrix - G.adj_matrix)))

100%|██████████| 379/379 [00:00<00:00, 801.86it/s]

Errors after erdos: 374


## Analysis 

In [11]:
# Identifying the components of the graph that are not reconstructed

A_prime = np.dot(Gstar_fixed.adj_matrix, Gstar_fixed.adj_matrix.T)
slots_of_error = np.argwhere(A != A_prime)
nodes_of_error = set()
for i, j in slots_of_error:
    nodes_of_error.add(i)
    nodes_of_error.add(j)

nodes_of_error = list(nodes_of_error)

components = {}
hubs = []
for node in nodes_of_error:
    if A[node, node] == A_prime[node, node]:
        hubs.append(node)


for hub in hubs:
    components[hub] = []
    for node in nodes_of_error:
        if A[hub, node] != A_prime[hub, node]:
            components[hub].append(node)


# print(components)

for hub, nodes in components.items():
    print(f" --- Hub ----: {hub}")
    for node in nodes:
        print(f"Node : {node}, degree in A : {A[node, node]}, degree in A_prime : {A_prime[node, node]}")

G_prime_1 = Gstar_fixed.copy()
G_prime_2 = Gstar_fixed.copy()

for hub, nodes in components.items():
    G_prime_1.add_edge((nodes[0], nodes[1]))
    G_prime_1.add_edge((nodes[2], nodes[3]))    

    G_prime_2.add_edge((nodes[0], nodes[3]))
    G_prime_2.add_edge((nodes[1], nodes[2]))

A_prime_1 = np.dot(G_prime_1.adj_matrix, G_prime_1.adj_matrix.T)
A_prime_2 = np.dot(G_prime_2.adj_matrix, G_prime_2.adj_matrix.T)


G_prime_1.to_txt(f"datasets/{dataset}_prime_1.txt")
G_prime_2.to_txt(f"datasets/{dataset}_prime_2.txt")
print("Errors after fixing:", np.sum(np.abs(A - A_prime_1)), np.sum(np.abs(A - A_prime_2)))


 --- Hub ----: 69
Node : 327, degree in A : 3, degree in A_prime : 2
Node : 328, degree in A : 3, degree in A_prime : 2
Node : 377, degree in A : 3, degree in A_prime : 2
Node : 378, degree in A : 3, degree in A_prime : 2
 --- Hub ----: 302
Node : 327, degree in A : 3, degree in A_prime : 2
Node : 328, degree in A : 3, degree in A_prime : 2
Node : 377, degree in A : 3, degree in A_prime : 2
Node : 378, degree in A : 3, degree in A_prime : 2
 --- Hub ----: 22
Node : 226, degree in A : 2, degree in A_prime : 1
Node : 227, degree in A : 2, degree in A_prime : 1
Node : 53, degree in A : 2, degree in A_prime : 1
Node : 54, degree in A : 2, degree in A_prime : 1
 --- Hub ----: 25
Node : 197, degree in A : 2, degree in A_prime : 1
Node : 294, degree in A : 2, degree in A_prime : 1
Node : 295, degree in A : 2, degree in A_prime : 1
Node : 250, degree in A : 2, degree in A_prime : 1
Errors after fixing: 0 0


In [7]:

dataset = "netscience"
G_prime_1 = Graph.from_txt(f"datasets/{dataset}_prime_1.txt")
G_prime_2 = Graph.from_txt(f"datasets/{dataset}_prime_2.txt")

100%|██████████| 379/379 [00:00<00:00, 773.18it/s]

Errors after spectral: 380
